In [1]:
# I think this is a util file that evaluates optimal elevation 
# per house across an ensemble
# So it will take in a dataframe -- representing our ensemble --
# and return the elevation height for each house
# So there needs to be a groupby on fd_id and return that way
# I think you want to pass the whole dataframe for speed

In [ ]:
# The gist of this is to evaluate costs & benefits for each increment
# of elevation from 3 to 10 feet (I think it's up to 10 feet - will
# double check)
# There are values for costs from the FEMA report that go in the config
# file
# This gets multipled by sq ft of the building
# For benefits, we'll do it "through" the DDF by moving along the
# x-axis of the DDF. We do this by changing the FFE, which we are
# treating under uncertainty. But we are not treating the elevation
# as an uncertain amount